In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import re
from imblearn.over_sampling import SMOTE

In [3]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
RCC = pd.read_csv('MergeData/all_sheets_merge1.csv')
pca = PCA(n_components=20)
sm = SMOTE(random_state = 12)
RCC['性別'] = RCC['性別'].map({'F':0,'M':1})
y = RCC['呼吸器成功脫離']
RCC = RCC.drop(['呼吸器成功脫離'], axis=1)

In [4]:
RCC.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in RCC.columns.values]
RCC = RCC.fillna(0)

In [5]:
reduced_data_pca = pca.fit_transform(RCC)
xgbc = XGBClassifier()
#np.any(np.isnan(RCC))
X_train, X_test, y_train, y_test = train_test_split(reduced_data_pca, y, test_size=0.25, random_state=33)

In [6]:
x_train_res,y_train_res = sm.fit_sample(X_train,y_train)

In [7]:
#rfc = RandomForestClassifier()
#rfc.fit(X_train,y_train)
#print('The accuracy of Random Forest Classifier on testing set:', rfc.score(X_test, y_test))

In [8]:
xgbc.fit(x_train_res, y_train_res)
print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))

The accuracy of eXtreme Gradient Boosting Classifier on testing set: 0.6363636363636364


In [9]:
kf = KFold(n_splits=10)
kf_scores = []
for train_index, test_index in kf.split(RCC):
    X_train, X_test = RCC.iloc[train_index], RCC.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    x_train_res,y_train_res = sm.fit_sample(X_train,y_train)
    xgbc.fit(x_train_res, y_train_res)
    kf_scores.append(xgbc.score(X_test, y_test))

In [10]:
print(kf_scores)
print(np.mean(kf_scores))
print(np.std(kf_scores))

[0.6363636363636364, 0.7727272727272727, 0.6363636363636364, 0.7727272727272727, 0.8181818181818182, 0.7727272727272727, 0.9090909090909091, 0.8181818181818182, 0.7272727272727273, 0.8636363636363636]
0.7727272727272727
0.08381404052084443
